In [ ]:
from datetime import datetime
import traceback
import re
import sys

from pymongo import MongoClient
import tweepy
from tweepy import API, OAuthHandler


# sys.path.append(r'/Users/laisky/repo/laisky/ramjet/ramjet/settings')
sys.path.append(r'/opt/configs/ramjet')
sys.path
import prd

mongo = MongoClient(
    f"mongodb://{prd.MONGO_USER}:{prd.MONGO_PASSWD}@{prd.MONGO_HOST}:{prd.MONGO_PORT}/{prd.MONGO_DB}",
)
tweets = mongo['twitter']['tweets']

auth = OAuthHandler(prd.CONSUMER_KEY, prd.CONSUMER_SECRET)
auth.set_access_token(prd.ACCESS_TOKEN, prd.ACCESS_TOKEN_SECRET)
api = API(auth, wait_on_rate_limit=True, parser=tweepy.parsers.JSONParser())
api.me()

In [ ]:
api.user_timeline(count=1)

In [ ]:
# delete tweets


def is_status_tobe_delete(tweet):
    if len(tweet.get('entities', {}).get("hashtags", [])) >= 1:
        return False
    
    if tweet['created_at'] < datetime(2017,6,5):
        return True

def delete():
#     last_tweets = api.user_timeline(count=1)
#     if not last_tweets:
#         return
    
#     print(last_tweets[0].created_at < datetime(2019,7,1))
#     print(last_tweets[0])
#     print(dir(last_tweets[0]))
#     return

#     current_id = last_tweets[0].id
    current_id = "379207474345046017"
    n = n_del = 1
#         tweets = api.user_timeline(max_id=current_id, count=1000)
    for tweet in tweets.find({'user.screen_name': 'ppcelery'}).sort("_id", -1):
        n += 1

        if is_status_tobe_delete(tweet):
            n_del += 1
            print("delete", tweet['id'],tweet['created_at'])
            try:
                api.destroy_status(tweet['id'])
            except Exception:
                traceback.print_exc()
        else:
            current_id = tweet['id']

        if n % 1000 == 0:
            print(">> scan {} statuses".format(n-1))
        if n_del % 100 == 0:
            print(">> delete {} statuses".format(n_del-1))


delete()

In [ ]:
# download image
import requests
from pathlib import Path

dirpath = r'/var/www/uploads/twitter'
# dirpath = r'/Users/laisky/Downloads'


def download_images_for_tweet(tweet):
    for img in tweet['entities']['media']:
        with requests.get(img['media_url_https']+":orig") as r:
            if r.status_code != 200:
                print(f"download error: [{r.status_code}]{r.content}")
                continue
                
            fpath = Path(dirpath, img['media_url_https'].split("/")[-1])
            if fpath.is_file():
                continue
                
            with open(fpath, 'wb') as f:
                f.write(r.content) 
                
            print("tweet img ok", tweet['id'], fpath)
            

def download_images():
    for tweet in tweets.find({"entities.media": {"$exists": 1}}, no_cursor_timeout=True).sort("_id", -1):
        download_images_for_tweet(tweet)
        
        
download_images()
        

In [ ]:
# download related tweets

def twitter_api_parser(tweet):
    reg_topic = re.compile(r"[\b|\s]#(\S+)")
    tweet['topics'] = reg_topic.findall(
        tweet['text'].replace('.', '_')
    )
    tweet['created_at'] = datetime.strptime(
        tweet['created_at'], '%a %b %d %H:%M:%S +0000 %Y'
    )

    # replace url
    t = tweet['text']
    if 't.co' in t:
        # parse entities media
        if 'media' in tweet['entities']:
            for media in tweet['entities']['media']:
                surl = media['url']
                eurl = media['media_url']
                t = t.replace(surl, eurl)

        # parse entities urls
        if 'urls' in tweet['entities']:
            for d in tweet['entities']['urls']:
                surl = d['url']
                eurl = d['expanded_url']
                t = t.replace(surl, eurl)

        tweet['text'] = t

    return tweet


def save_relate_tweets(status):
    related_ids = []
    status.get("in_reply_to_status_id") and related_ids.append(status['in_reply_to_status_id'])
    status.get("retweeted_status") and related_ids.append(status['retweeted_status']['id'])
    status.get("quoted_status") and related_ids.append(status['quoted_status']['id'])
    related_ids = filter(lambda id_: not tweets.find_one({"id": id_}), related_ids)
    
    for id_ in related_ids:
        try:
            docu = api.get_status(id_)
        except Exception as err:
            print(f"load tweet {id_} got error: {err}")
        else:
            print(f"save tweet [{docu['user']['screen_name']}]{docu['id']}")
            save_tweet(docu)
            save_relate_tweets(docu)
            
def save_tweet(docu):
    docu = twitter_api_parser(docu)
    tweets.update_one(
        {'id': docu['id']},
        {'$set': docu},
        upsert=True
    )

def download_relate_tweets():
    for tweet in tweets.find(no_cursor_timeout=True).sort("_id", -1):
        save_relate_tweets(tweet)

        
download_relate_tweets()
